# **AI-Powered Fleet Optimization & Road Incident Prediction for Smart Transportation**

## **Objective**
Develop an end-to-end ML system that:
1.	Predicts potential route incidents (accidents, breakdowns, slowdowns).
2.	Optimizes fleet scheduling & vehicle assignment in real time.
3.	Automatically re-routes vehicles during disruptions.
This is an enterprise-level ML pipeline used by large-scale logistics companies.


# **Phase 1: Real-Time Data Integration Layer**

This phase simulates real-time fleet data such as GPS location, vehicle speed,
engine health, traffic conditions, and weather data. The generated stream
acts as the live input source for downstream ML models.



In [1]:
import os

phase1_dir = "phase1_realtime_stream"

os.makedirs(phase1_dir, exist_ok=True)

print("Folder created:", phase1_dir)


Folder created: phase1_realtime_stream


### **Configuration Management**
Defines centralized configuration parameters such as data paths,
streaming frequency, and simulation constants.

In [3]:
%%writefile phase1_realtime_stream/config.py
# =========================
# Streaming Configuration
# =========================

STREAM_INTERVAL = 2  # seconds

# Starting GPS location (Hyderabad)
START_LAT = 17.3850
START_LON = 78.4867

SPEED_RANGES = {
    "highway": (60, 100),
    "urban": (20, 50),
    "rural": (30, 60)
}

WEATHER_TYPES = ["clear", "rain", "fog"]

WEATHER_RISK = {
    "clear": 0.1,
    "rain": 0.6,
    "fog": 0.8
}


Writing phase1_realtime_stream/config.py


### **Vehicle Master Dataset**
Stores static vehicle information including capacity, engine type,
and vehicle category used during optimization and prediction.

In [4]:
%%writefile phase1_realtime_stream/vehicle_master.csv
vehicle_id,vehicle_type,capacity,engine_type
V101,Truck,1000,Diesel
V102,Van,700,Petrol
V103,Truck,1200,Diesel
V104,Van,650,Petrol
V105,Truck,1500,Diesel
V106,Van,600,Petrol
V107,Truck,1300,Diesel
V108,Van,750,Petrol
V109,Truck,1400,Diesel
V110,Van,800,Petrol
V111,Truck,1000,Diesel
V112,Van,700,Petrol
V113,Truck,1200,Diesel
V114,Van,650,Petrol
V115,Truck,1500,Diesel
V116,Van,600,Petrol
V117,Truck,1300,Diesel
V118,Van,750,Petrol
V119,Truck,1400,Diesel
V120,Van,800,Petrol
V121,Truck,1000,Diesel
V122,Van,700,Petrol
V123,Truck,1200,Diesel
V124,Van,650,Petrol
V125,Truck,1500,Diesel

Writing phase1_realtime_stream/vehicle_master.csv


### **Past Accident Database**
Contains historical accident coordinates and severity levels.
Used for training incident prediction models.


In [5]:
%%writefile phase1_realtime_stream/past_accidents.csv
latitude,longitude,severity
17.3855,78.4869,High
17.3872,78.4881,Medium
17.3901,78.4905,Low



Writing phase1_realtime_stream/past_accidents.csv


### **Real-Time Fleet Data Generator**
Simulates continuous streaming data for:
- GPS coordinates
- Vehicle speed
- Engine health
- Driver behavior
- Traffic and weather conditions

Acts as a substitute for Kafka in this implementation.


In [9]:
%%writefile phase1_realtime_stream/data_generator.py
import os
import time
import random
import json
from datetime import datetime

import pandas as pd
import numpy as np
from geopy.distance import geodesic
import pytz

from config import *

# =====================================================
# TIMEZONE CONFIGURATION
# =====================================================
IST = pytz.timezone("Asia/Kolkata")

# =====================================================
# PATH CONFIGURATION
# =====================================================
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
PROJECT_ROOT = os.path.dirname(BASE_DIR)

OUTPUT_DIR = os.path.join(PROJECT_ROOT, "phase5_dashboard", "data")
os.makedirs(OUTPUT_DIR, exist_ok=True)

CSV_PATH = os.path.join(OUTPUT_DIR, "live_stream.csv")
if os.path.exists(CSV_PATH):
    os.remove(CSV_PATH)

# =====================================================
# LOAD STATIC DATA
# =====================================================
vehicles = pd.read_csv(os.path.join(BASE_DIR, "vehicle_master.csv"))
accidents = pd.read_csv(os.path.join(BASE_DIR, "past_accidents.csv"))

# =====================================================
# INITIALIZE VEHICLE POSITIONS
# =====================================================
vehicle_positions = {
    v.vehicle_id: [START_LAT, START_LON]
    for _, v in vehicles.iterrows()
}

# =====================================================
# CONTROLLED RISK SIMULATION
# These vehicles intentionally produce abnormal patterns
# to test incident prediction and rerouting logic.
# =====================================================
HIGH_RISK_VEHICLES = random.sample(list(vehicle_positions.keys()), k=3)
FORCED_REROUTE_VEHICLES = random.sample(list(vehicle_positions.keys()), k=2)

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def move_gps(lat, lon):
    """Simulates small GPS movement."""
    return (
        round(lat + random.uniform(-0.0005, 0.0005), 6),
        round(lon + random.uniform(-0.0005, 0.0005), 6)
    )

def road_type():
    """Randomly selects road type with realistic distribution."""
    return random.choices(
        ["highway", "urban", "rural"],
        weights=[0.4, 0.4, 0.2]
    )[0]

def traffic_index():
    """Returns traffic congestion score between 0 and 1."""
    return round(random.uniform(0.1, 0.9), 2)

def accident_risk(lat, lon):
    """Calculates historical accident risk based on proximity."""
    risk = 0.0
    for _, row in accidents.iterrows():
        distance_km = geodesic((lat, lon), (row.latitude, row.longitude)).km
        if distance_km < 1:
            risk += {"High": 0.6, "Medium": 0.3, "Low": 0.1}[row.severity]
    return round(min(risk, 1.0), 2)

def engine_health(vehicle_id):
    """Simulates engine condition and failure risk."""
    if vehicle_id in FORCED_REROUTE_VEHICLES:
        return {
            "engine_temp": 120,
            "fuel_level": 5,
            "oil_pressure": 15,
            "engine_risk": 0.95
        }

    if vehicle_id in HIGH_RISK_VEHICLES or random.random() < 0.25:
        return {
            "engine_temp": random.randint(110, 120),
            "fuel_level": random.randint(5, 15),
            "oil_pressure": random.randint(15, 25),
            "engine_risk": 0.8
        }

    return {
        "engine_temp": random.randint(70, 100),
        "fuel_level": random.randint(20, 100),
        "oil_pressure": random.randint(30, 80),
        "engine_risk": 0.2
    }

def driver_behavior(vehicle_id):
    """Simulates driver driving patterns."""
    if vehicle_id in FORCED_REROUTE_VEHICLES:
        return {
            "hard_brake": 1,
            "rapid_acceleration": 1,
            "idle_time": 400,
            "driver_risk": 0.9
        }

    if vehicle_id in HIGH_RISK_VEHICLES or random.random() < 0.25:
        return {
            "hard_brake": 1,
            "rapid_acceleration": 1,
            "idle_time": random.randint(250, 400),
            "driver_risk": 0.7
        }

    return {
        "hard_brake": random.choice([0, 1]),
        "rapid_acceleration": random.choice([0, 1]),
        "idle_time": random.randint(0, 200),
        "driver_risk": 0.2
    }

def weather_data(vehicle_id):
    """Simulates weather conditions and associated risk."""
    if vehicle_id in FORCED_REROUTE_VEHICLES:
        weather = random.choice(["rain", "fog"])
        return {
            "weather": weather,
            "temperature": random.randint(30, 40),
            "weather_risk": 0.9
        }

    weather = (
        random.choice(["rain", "fog"])
        if vehicle_id in HIGH_RISK_VEHICLES and random.random() < 0.7
        else random.choice(WEATHER_TYPES)
    )

    return {
        "weather": weather,
        "temperature": random.randint(15, 40),
        "weather_risk": WEATHER_RISK[weather]
    }

def nearby_accident_severity(vehicle_id):
    """Returns nearby accident severity level."""
    if vehicle_id in FORCED_REROUTE_VEHICLES:
        return "High"

    if vehicle_id in HIGH_RISK_VEHICLES and random.random() < 0.7:
        return np.random.choice(["Medium", "High"], p=[0.3, 0.7])

    return np.random.choice(
        ["None", "Low", "Medium", "High"],
        p=[0.5, 0.2, 0.2, 0.1]
    )

# =====================================================
# EVENT GENERATOR
# =====================================================

def generate_event(vehicle):
    """Generates a single telemetry event for a vehicle."""
    lat, lon = vehicle_positions[vehicle.vehicle_id]
    lat, lon = move_gps(lat, lon)
    vehicle_positions[vehicle.vehicle_id] = [lat, lon]

    road = road_type()
    speed = random.randint(*SPEED_RANGES[road])

    return {
        "event_id": f"{vehicle.vehicle_id}_{datetime.now().timestamp()}",
        "timestamp": datetime.now(IST).isoformat(),
        "vehicle_id": vehicle.vehicle_id,
        "vehicle_type": vehicle.vehicle_type,
        "capacity": vehicle.capacity,
        "latitude": lat,
        "longitude": lon,
        "road_type": road,
        "speed": speed,
        "traffic_index": traffic_index(),
        **engine_health(vehicle.vehicle_id),
        **driver_behavior(vehicle.vehicle_id),
        **weather_data(vehicle.vehicle_id),
        "nearby_accident_severity": nearby_accident_severity(vehicle.vehicle_id),
        "historical_accident_risk": accident_risk(lat, lon)
    }

# =====================================================
# STREAMING LOOP
# =====================================================

def stream_data():
    """Continuously streams real-time fleet data."""
    print("[INFO] Phase-1 real-time fleet data streaming started.")

    try:
        while True:
            for _, vehicle in vehicles.iterrows():
                event = generate_event(vehicle)
                pd.DataFrame([event]).to_csv(
                    CSV_PATH,
                    mode="a",
                    header=not os.path.exists(CSV_PATH),
                    index=False
                )
                print(f"[DATA] Event generated for vehicle {vehicle.vehicle_id}")
                time.sleep(STREAM_INTERVAL)

    except KeyboardInterrupt:
        print("\n[INFO] Streaming stopped manually by user (Ctrl+C).")

    except Exception as e:
        print(f"[ERROR] Unexpected error occurred: {e}")

    finally:
        print("[INFO] Phase-1 streaming service shut down gracefully.")

if __name__ == "__main__":
    stream_data()


Overwriting phase1_realtime_stream/data_generator.py


# **PHASE-2 — INCIDENT PREDICTION SYSTEM**


This phase builds ML models to predict accident risk, vehicle breakdowns,
route slowdowns, and potential disruptions using real-time features.


In [10]:
import os

phase2_dir = "phase2_incident_prediction"

os.makedirs(phase2_dir, exist_ok=True)

print("Folder created:", phase2_dir)


Folder created: phase2_incident_prediction


### **Incident Prediction Model Training**
Trains multiple ML models including:
- Random Forest
- XGBoost
- CatBoost
- LSTM/GRU (sequence model)

Evaluates models using precision, recall, F1-score, ROC-AUC,
and false negative rate.


In [17]:
%%writefile phase2_incident_prediction/train_incident_models.py

import os
import joblib
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

# =====================================================
# PATH CONFIGURATION
# =====================================================
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATA_DIR = os.path.join(BASE_DIR, "data")
MODEL_DIR = os.path.join(BASE_DIR, "models")

MODEL_VERSION = datetime.now().strftime("v%Y%m%d_%H%M%S")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# =====================================================
# METRICS FUNCTION
# =====================================================
def compute_metrics(y_true, y_pred, y_prob):
    fnr = ((y_true == 1) & (y_pred == 0)).sum() / max(1, (y_true == 1).sum())
    return {
        "Precision": round(precision_score(y_true, y_pred, zero_division=0), 3),
        "Recall": round(recall_score(y_true, y_pred, zero_division=0), 3),
        "F1": round(f1_score(y_true, y_pred, zero_division=0), 3),
        "ROC-AUC": round(roc_auc_score(y_true, y_prob), 3),
        "False Negative Rate": round(fnr, 3)
    }

# =====================================================
# SYNTHETIC DATA GENERATION (SIMULATION)
# =====================================================
def generate_data(n=10000):
    df = pd.DataFrame({
        "speed": np.random.randint(10, 100, n),
        "traffic_index": np.random.uniform(0.1, 1.0, n),
        "engine_temp": np.random.randint(70, 120, n),
        "fuel_level": np.random.randint(5, 100, n),
        "oil_pressure": np.random.randint(15, 80, n),
        "hard_brake": np.random.choice([0, 1], n),
        "rapid_acceleration": np.random.choice([0, 1], n),
        "idle_time": np.random.randint(0, 400, n),
        "road_type": np.random.choice(["highway", "urban", "rural"], n),
        "weather": np.random.choice(["clear", "rain", "fog"], n),
        "temperature": np.random.randint(10, 45, n),
        "nearby_accident_severity": np.random.choice(
            ["None", "Low", "Medium", "High"], n
        )
    })

    df["accident_risk"] = (
        (df["traffic_index"] > 0.7) &
        (df["weather"].isin(["rain", "fog"])) &
        (df["hard_brake"] == 1)
    ).astype(int)

    df["breakdown_risk"] = (
        (df["engine_temp"] > 105) |
        (df["fuel_level"] < 15)
    ).astype(int)

    df["slowdown_risk"] = (
        (df["speed"] < 25) &
        (df["traffic_index"] > 0.6)
    ).astype(int)

    df["route_blockage_risk"] = (
        (df["traffic_index"] > 0.85) &
        (df["nearby_accident_severity"] == "High")
    ).astype(int)

    return df

print("[INFO] Generating training data...")
df = generate_data()
df.to_csv(os.path.join(DATA_DIR, "incident_training_data.csv"), index=False)

# =====================================================
# PREPROCESSING
# =====================================================
targets = [
    "accident_risk",
    "breakdown_risk",
    "slowdown_risk",
    "route_blockage_risk"
]

X = df.drop(columns=targets)
y = df[targets]

categorical_cols = ["road_type", "weather", "nearby_accident_severity"]
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

joblib.dump(encoders, os.path.join(MODEL_DIR, f"encoders_{MODEL_VERSION}.pkl"))
joblib.dump(scaler, os.path.join(MODEL_DIR, f"scaler_{MODEL_VERSION}.pkl"))

# =====================================================
# TRAIN / TEST SPLIT
# =====================================================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =====================================================
# TREE-BASED MODELS
# =====================================================
tree_models = {}

for risk in targets:
    print(f"[INFO] Training tree-based models for target: {risk}")

    rf = RandomForestClassifier(
        n_estimators=200,
        class_weight="balanced",
        random_state=42
    )

    xgb = XGBClassifier(
        eval_metric="logloss",
        scale_pos_weight=(y_train[risk] == 0).sum() / max(1, (y_train[risk] == 1).sum())
    )

    cat = CatBoostClassifier(iterations=150, verbose=0)

    rf.fit(X_train, y_train[risk])
    xgb.fit(X_train, y_train[risk])
    cat.fit(X_train, y_train[risk])

    prob = rf.predict_proba(X_test)[:, 1]
    pred = (prob > 0.5).astype(int)

    print("[METRICS]", compute_metrics(y_test[risk], pred, prob))

    tree_models[risk] = {
        "rf": rf,
        "xgb": xgb,
        "cat": cat
    }

joblib.dump(
    tree_models,
    os.path.join(MODEL_DIR, f"tree_models_{MODEL_VERSION}.pkl")
)

# =====================================================
# LSTM SEQUENCE MODELS (TEMPORAL)
# =====================================================
SEQ_LEN = 5
N_FEATURES = X_scaled.shape[1]

def build_sequences(X, y):
    X_seq, y_seq = [], []
    for i in range(len(X) - SEQ_LEN):
        X_seq.append(X[i:i + SEQ_LEN])
        y_seq.append(y[i + SEQ_LEN])
    return np.array(X_seq), np.array(y_seq)

lstm_models = {}

for risk in targets:
    print(f"[INFO] Training LSTM model for target: {risk}")

    X_seq, y_seq = build_sequences(X_scaled, y[risk].values)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_seq, y_seq, test_size=0.2, random_state=42
    )

    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_tr),
        y=y_tr
    )
    class_weight_dict = dict(enumerate(class_weights))

    model = Sequential([
        Input(shape=(SEQ_LEN, N_FEATURES)),
        LSTM(64, return_sequences=True),
        Dropout(0.3),
        LSTM(32),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["Recall"]
    )

    model.fit(
        X_tr,
        y_tr,
        epochs=15,
        batch_size=64,
        validation_split=0.2,
        class_weight=class_weight_dict,
        callbacks=[EarlyStopping(patience=4, restore_best_weights=True)],
        verbose=0
    )

    y_prob = model.predict(X_te).ravel()
    y_pred = (y_prob > 0.4).astype(int)  # safety-oriented threshold

    print("[METRICS]", compute_metrics(y_te, y_pred, y_prob))

    lstm_models[risk] = model

joblib.dump(
    lstm_models,
    os.path.join(MODEL_DIR, f"lstm_models_{MODEL_VERSION}.pkl")
)

print("[SUCCESS] Phase-2 model training completed successfully.")


Overwriting phase2_incident_prediction/train_incident_models.py


### **Automated Model Retraining**
Implements scheduled retraining logic to ensure models remain
up-to-date with evolving traffic and fleet behavior.


In [18]:
%%writefile phase2_incident_prediction/auto_retrain.py
import pytz
import subprocess
from datetime import datetime
IST = pytz.timezone("Asia/Kolkata")

print("[INFO] Automated retraining job triggered")
print("[INFO] Timestamp:", datetime.now(IST).isoformat())

subprocess.run(
    ["python", "phase2_incident_prediction/train_incident_models.py"],
    check=True
)


Writing phase2_incident_prediction/auto_retrain.py


### **Data Drift Monitoring**
Detects distribution shifts in GPS patterns, incident frequency,
and traffic behavior to trigger retraining when necessary.


In [15]:
%%writefile phase2_incident_prediction/drift_monitor.py
import pandas as pd
import numpy as np

BASELINE = "phase2_incident_prediction/data/incident_training_data.csv"
LIVE = "phase5_dashboard/data/live_stream.csv"

DRIFT_THRESHOLD = 0.3

baseline = pd.read_csv(BASELINE)
live = pd.read_csv(LIVE).tail(500)

features = [
    "speed",
    "traffic_index",
    "engine_temp",
    "fuel_level",
    "idle_time"
]

print("[INFO] Drift monitoring report")

for f in features:
    base_mean = baseline[f].mean()
    live_mean = live[f].mean()

    drift_score = abs(live_mean - base_mean) / (base_mean + 1e-6)

    if drift_score > DRIFT_THRESHOLD:
        print(f"[WARNING] Drift detected in {f}: score={drift_score:.2f}")
    else:
        print(f"[OK] {f} stable (score={drift_score:.2f})")


Writing phase2_incident_prediction/drift_monitor.py


### **Real-Time Incident Inference**
Consumes live data streams and produces real-time incident
probability predictions used by routing and optimization engines.


In [19]:
%%writefile phase1_realtime_stream/phase2_live_inference.py
import os
import time
import joblib
import numpy as np
import pandas as pd

from datetime import datetime
from collections import deque

# =====================================================
# PATH CONFIGURATION
# =====================================================
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

STREAM_FILE = os.path.join(
    BASE_DIR, "..", "phase5_dashboard", "data", "live_stream.csv"
)

OUTPUT_FILE = os.path.join(
    BASE_DIR, "..", "phase5_dashboard", "data", "live_predictions.csv"
)

MODEL_DIR = os.path.join(
    BASE_DIR, "..", "phase2_incident_prediction", "models"
)

os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# =====================================================
# MODEL LOADING
# =====================================================
def load_latest(prefix: str):
    files = [f for f in os.listdir(MODEL_DIR) if f.startswith(prefix)]
    if not files:
        raise FileNotFoundError(f"No models found for prefix: {prefix}")
    latest = sorted(files)[-1]
    return joblib.load(os.path.join(MODEL_DIR, latest))

print("[INFO] Loading trained models...")

tree_models = load_latest("tree_models")
lstm_models = load_latest("lstm_models")
encoders = load_latest("encoders")
scaler = load_latest("scaler")

print("[SUCCESS] Models loaded successfully")

# =====================================================
# FEATURE DEFINITIONS
# =====================================================
categorical_cols = [
    "road_type",
    "weather",
    "nearby_accident_severity"
]

feature_cols = [
    "speed", "traffic_index", "engine_temp", "fuel_level", "oil_pressure",
    "hard_brake", "rapid_acceleration", "idle_time",
    "road_type", "weather", "temperature",
    "nearby_accident_severity"
]

risk_types = list(tree_models.keys())

# =====================================================
# LSTM SEQUENCE CONFIGURATION
# =====================================================
SEQ_LEN = 5
vehicle_sequences = {}   # vehicle_id -> deque

# =====================================================
# PREPROCESSING FUNCTION
# =====================================================
def preprocess(df: pd.DataFrame) -> np.ndarray:
    """
    Ensures schema consistency, encodes categoricals,
    and applies standard scaling.
    """
    df = df.copy()

    # Fill missing columns
    for col in feature_cols:
        if col not in df.columns:
            df[col] = "None" if col in categorical_cols else 0

    # Encode categoricals safely
    for col in categorical_cols:
        df[col] = df[col].fillna("None")
        known = set(encoders[col].classes_)
        df[col] = df[col].apply(lambda x: x if x in known else "None")
        df[col] = encoders[col].transform(df[col])

    return scaler.transform(df[feature_cols])

# =====================================================
# STREAMING INFERENCE LOOP
# =====================================================
print("[INFO] Phase-2 Live Inference Started")

last_row = 0

while True:
    try:
        if not os.path.exists(STREAM_FILE):
            time.sleep(2)
            continue

        df = pd.read_csv(STREAM_FILE)

        if len(df) <= last_row:
            time.sleep(2)
            continue

        new_data = df.iloc[last_row:].reset_index(drop=True)
        last_row = len(df)

        X_live = preprocess(new_data)

        output = {
            "timestamp": new_data["timestamp"],
            "vehicle_id": new_data["vehicle_id"]
        }

        # -------------------------------------------------
        # LSTM SEQUENCE PREDICTIONS
        # -------------------------------------------------
        lstm_preds = {}

        for i, vid in enumerate(new_data["vehicle_id"]):
            vehicle_sequences.setdefault(vid, deque(maxlen=SEQ_LEN))
            vehicle_sequences[vid].append(X_live[i])

            if len(vehicle_sequences[vid]) == SEQ_LEN:
                seq = np.array(vehicle_sequences[vid]).reshape(1, SEQ_LEN, -1)
                lstm_preds[vid] = {
                    risk: lstm_models[risk].predict(seq, verbose=0)[0][0]
                    for risk in risk_types
                }

        # -------------------------------------------------
        # TREE + LSTM ENSEMBLE
        # -------------------------------------------------
        for risk in risk_types:
            rf = tree_models[risk]["rf"]
            xgb = tree_models[risk]["xgb"]
            cat = tree_models[risk]["cat"]

            tree_prob = (
                0.4 * rf.predict_proba(X_live)[:, 1] +
                0.4 * xgb.predict_proba(X_live)[:, 1] +
                0.2 * cat.predict_proba(X_live)[:, 1]
            )

            final_prob = []

            for i, vid in enumerate(new_data["vehicle_id"]):
                lstm_p = lstm_preds.get(vid, {}).get(risk, tree_prob[i])
                final_prob.append(0.7 * tree_prob[i] + 0.3 * lstm_p)

            p = np.array(final_prob)

            # -------------------------------------------------
            # SAFETY ESCALATION LOGIC
            # -------------------------------------------------
            escalation = (
                (new_data["traffic_index"] > 0.85).astype(int) +
                (new_data["engine_temp"] > 105).astype(int) +
                (new_data["weather"].isin(["rain", "fog"])).astype(int) +
                (new_data["nearby_accident_severity"] == "High").astype(int)
            )

            p = np.where(escalation >= 2, np.maximum(p, 0.75), p)

            output[risk] = np.round(p, 3)
            output[f"{risk}_confidence"] = np.round(p, 3)

        # -------------------------------------------------
        # SAVE OUTPUT
        # -------------------------------------------------
        pd.DataFrame(output).to_csv(
            OUTPUT_FILE,
            mode="a",
            header=not os.path.exists(OUTPUT_FILE),
            index=False
        )

        print(
            f"[INFO] {len(new_data)} predictions generated @ "
            f"{datetime.now().strftime('%H:%M:%S')}"
        )

        time.sleep(2)

    except KeyboardInterrupt:
        print("\n[SHUTDOWN] Live inference stopped safely by user")
        break

    except Exception as e:
        print("[ERROR] Inference failure:", str(e))
        time.sleep(3)


Overwriting phase1_realtime_stream/phase2_live_inference.py


# **PHASE-3: Fleet Scheduling Optimization (Genetic Algorithm)**

This phase optimizes vehicle-to-task assignment while minimizing
fuel cost, delay, and idle time under operational constraints.


In [20]:

phase3_dir = "phase3_fleet_optimization"

os.makedirs(phase3_dir, exist_ok=True)

print("Folder created:", phase3_dir)


Folder created: phase3_fleet_optimization


### **Fleet Optimization Engine**
Implements a Genetic Algorithm / OR-Tools based Vehicle Routing Problem (VRP)
solver that accounts for:
- Vehicle capacity
- Driver shift limits
- Traffic congestion
- Delivery time windows


In [21]:
%%writefile phase3_fleet_optimization/fleet_optimizer.py
"""

This module assigns delivery tasks to vehicles using a Genetic Algorithm
while incorporating real-time risk predictions and GPS data. Vehicles
with high predicted risk are flagged for rerouting.

Designed for demo, dashboard integration, and enterprise-style ML pipelines.
"""

import os
import time
import random
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

# -------------------------------------------------------------------
# Timezone Configuration
# -------------------------------------------------------------------
IST = pytz.timezone("Asia/Kolkata")

# -------------------------------------------------------------------
# Path Configuration
# -------------------------------------------------------------------
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

VEHICLE_MASTER = os.path.join(
    BASE_DIR, "phase1_realtime_stream", "vehicle_master.csv"
)

LIVE_STREAM = os.path.join(
    BASE_DIR, "phase5_dashboard", "data", "live_stream.csv"
)

PREDICTIONS_FILE = os.path.join(
    BASE_DIR, "phase5_dashboard", "data", "live_predictions.csv"
)

OUTPUT_DIR = os.path.join(
    BASE_DIR, "phase5_dashboard", "data"
)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------------------------------------------------
# System Parameters
# -------------------------------------------------------------------
RUN_INTERVAL = 30  # seconds

RISK_THRESHOLD = 0.65
CRITICAL_THRESHOLD = 0.85

DEFAULT_DEST = {"lat": 17.4500, "lon": 78.5000}

# Demo mode forces rerouting for dashboard visibility
DEMO_MODE = True
DEMO_REROUTE_COUNT = 10

# -------------------------------------------------------------------
# Delivery Task Definitions (Synthetic)
# -------------------------------------------------------------------
TASKS = [
    {"task_id": 0, "distance": 40, "demand": 30, "traffic": 1.2, "deadline": 5},
    {"task_id": 1, "distance": 60, "demand": 50, "traffic": 1.5, "deadline": 7},
    {"task_id": 2, "distance": 30, "demand": 20, "traffic": 1.1, "deadline": 4},
    {"task_id": 3, "distance": 80, "demand": 40, "traffic": 1.6, "deadline": 8},
]

# -------------------------------------------------------------------
# Genetic Algorithm Helpers
# -------------------------------------------------------------------
def create_chromosome(num_tasks, num_vehicles):
    return [random.randint(0, num_vehicles - 1) for _ in range(num_tasks)]

def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return parent1[:point] + parent2[point:]

def mutate(chromosome, num_vehicles, rate=0.15):
    for i in range(len(chromosome)):
        if random.random() < rate:
            chromosome[i] = random.randint(0, num_vehicles - 1)
    return chromosome

def tournament_selection(population, fitness_fn, k=3):
    return min(random.sample(population, k), key=fitness_fn)

# -------------------------------------------------------------------
# Core Optimization Logic
# -------------------------------------------------------------------
def run_fleet_optimization():
    """
    Executes one optimization cycle:
    - Loads vehicles and live GPS data
    - Integrates ML risk predictions
    - Runs GA-based task assignment
    - Flags vehicles requiring rerouting
    - Saves optimized fleet schedule
    """

    # Load vehicle master
    vehicles_df = pd.read_csv(VEHICLE_MASTER)

    if "shift_hours" not in vehicles_df.columns:
        vehicles_df["shift_hours"] = 8

    vehicles_df["risk_score"] = 0.0
    vehicles_df["reroute_required"] = False
    vehicles_df["reroute_reason"] = ""

    # -------------------------------------------------------------------
    # Live GPS Integration
    # -------------------------------------------------------------------
    if os.path.exists(LIVE_STREAM):
        live_df = pd.read_csv(LIVE_STREAM)
        latest = live_df.groupby("vehicle_id").last().reset_index()

        vehicles_df = vehicles_df.merge(
            latest[["vehicle_id", "latitude", "longitude"]],
            on="vehicle_id",
            how="left"
        )
    else:
        vehicles_df["latitude"] = 17.3850
        vehicles_df["longitude"] = 78.4867

    # -------------------------------------------------------------------
    # Phase-2 Risk Prediction Integration
    # -------------------------------------------------------------------
    if os.path.exists(PREDICTIONS_FILE):
        pred_df = pd.read_csv(PREDICTIONS_FILE)
        latest_pred = pred_df.groupby("vehicle_id").last().reset_index()

        vehicles_df = (
            vehicles_df.merge(latest_pred, on="vehicle_id", how="left")
            .fillna(0)
        )

        for idx, row in vehicles_df.iterrows():
            accident = row.get("accident_risk", 0)
            breakdown = row.get("breakdown_risk", 0)
            blockage = row.get("route_blockage_risk", 0)

            risk_score = round(
                0.5 * accident + 0.3 * breakdown + 0.2 * blockage, 3
            )

            vehicles_df.at[idx, "risk_score"] = risk_score

            reasons = []
            if risk_score >= RISK_THRESHOLD:
                reasons.append("High composite risk")
            if accident >= CRITICAL_THRESHOLD:
                reasons.append("Critical accident risk")
            if breakdown >= CRITICAL_THRESHOLD:
                reasons.append("Critical breakdown risk")
            if blockage >= CRITICAL_THRESHOLD:
                reasons.append("Critical route blockage")

            if reasons:
                vehicles_df.at[idx, "reroute_required"] = True
                vehicles_df.at[idx, "reroute_reason"] = "; ".join(reasons)

    # -------------------------------------------------------------------
    # Demo Override (for visualization)
    # -------------------------------------------------------------------
    if DEMO_MODE:
        demo_targets = vehicles_df.sort_values(
            "risk_score", ascending=False
        ).head(DEMO_REROUTE_COUNT)

        for i, idx in enumerate(demo_targets.index):
            if i < len(demo_targets) * 0.5:
                vehicles_df.at[idx, "risk_score"] = 0.55
                vehicles_df.at[idx, "reroute_required"] = False
                vehicles_df.at[idx, "reroute_reason"] = "Demo: Moderate risk"
            else:
                vehicles_df.at[idx, "risk_score"] = 0.88
                vehicles_df.at[idx, "reroute_required"] = True
                vehicles_df.at[idx, "reroute_reason"] = "Demo: Critical risk"

    # Destination
    vehicles_df["dest_lat"] = DEFAULT_DEST["lat"]
    vehicles_df["dest_lon"] = DEFAULT_DEST["lon"]

    vehicles = vehicles_df.to_dict("records")

    # -------------------------------------------------------------------
    # Fitness Function
    # -------------------------------------------------------------------
    def fitness(chromosome):
        cost = 0
        load = {v["vehicle_id"]: 0 for v in vehicles}

        for i, vehicle_idx in enumerate(chromosome):
            task = TASKS[i]
            vehicle = vehicles[vehicle_idx]

            load[vehicle["vehicle_id"]] += task["demand"]
            cost += task["distance"] * 0.05
            cost += vehicle["risk_score"] * 120

        cost += np.var(list(load.values())) * 2
        return cost

    # -------------------------------------------------------------------
    # Run Genetic Algorithm
    # -------------------------------------------------------------------
    population = [
        create_chromosome(len(TASKS), len(vehicles))
        for _ in range(40)
    ]

    for _ in range(60):
        population = [
            mutate(
                crossover(
                    tournament_selection(population, fitness),
                    tournament_selection(population, fitness)
                ),
                len(vehicles)
            )
            for _ in range(len(population))
        ]

    best_solution = min(population, key=fitness)

    schedule = pd.DataFrame({
        "task_id": [t["task_id"] for t in TASKS],
        "vehicle_id": [vehicles[i]["vehicle_id"] for i in best_solution],
        "risk_score": [vehicles[i]["risk_score"] for i in best_solution],
        "reroute_required": [vehicles[i]["reroute_required"] for i in best_solution],
        "reroute_reason": [vehicles[i]["reroute_reason"] for i in best_solution],
        "latitude": [vehicles[i]["latitude"] for i in best_solution],
        "longitude": [vehicles[i]["longitude"] for i in best_solution],
        "dest_lat": [vehicles[i]["dest_lat"] for i in best_solution],
        "dest_lon": [vehicles[i]["dest_lon"] for i in best_solution],
        "timestamp": datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S")
    })

    schedule.to_csv(
        os.path.join(OUTPUT_DIR, "fleet_schedule.csv"),
        index=False
    )

    flagged = schedule["reroute_required"].sum()
    if flagged:
        print(f"[INFO] {flagged} vehicle(s) flagged for rerouting.")
    else:
        print("[INFO] Fleet operating within safe risk thresholds.")

# -------------------------------------------------------------------
# Continuous Execution Loop
# -------------------------------------------------------------------
if __name__ == "__main__":
    print("[SYSTEM] Phase-3 Fleet Scheduling Optimization started.")

    try:
        while True:
            run_fleet_optimization()
            time.sleep(RUN_INTERVAL)
    except KeyboardInterrupt:
        print("\n[SYSTEM] Phase-3 optimization stopped by user.")


Writing phase3_fleet_optimization/fleet_optimizer.py


# **PHASE-4 — DYNAMIC REROUTING SYSTEM**

This phase dynamically reroutes vehicles when high-risk incidents
are predicted to ensure safety and on-time delivery.


In [22]:
phase4_dir = "phase4_dynamic_rerouting"

os.makedirs(phase4_dir, exist_ok=True)

print("Folder created:", phase4_dir)

Folder created: phase4_dynamic_rerouting


### **Intelligent Route Recalculation**
Uses OSRM / OpenRouteService APIs to:
- Identify safer alternative routes
- Update ETA dynamically
- Log rerouting events for monitoring


In [23]:
%%writefile phase4_dynamic_rerouting/dynamic_rerouting.py
"""
This module performs dynamic rerouting strictly based on Phase-3 fleet
optimization output. Vehicles flagged for rerouting are assigned a new
destination, ETA is recalculated, and reroute events are logged.

Designed for real-time dashboards and intelligent transportation systems.
"""

import os
import time
import random
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import folium
import pytz

# -------------------------------------------------------------------
# Timezone Configuration
# -------------------------------------------------------------------
IST = pytz.timezone("Asia/Kolkata")

# -------------------------------------------------------------------
# Path Configuration
# -------------------------------------------------------------------
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

FLEET_FILE = os.path.join(
    BASE_DIR, "phase5_dashboard", "data", "fleet_schedule.csv"
)

OUTPUT_DIR = os.path.join(
    BASE_DIR, "phase5_dashboard", "data"
)

REROUTE_LOG = os.path.join(
    OUTPUT_DIR, "rerouting_logs.csv"
)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------------------------------------------------
# System Parameters
# -------------------------------------------------------------------
UPDATE_INTERVAL = 30          # seconds
MAX_DEST_SHIFT = 0.004        # realistic destination deviation

# -------------------------------------------------------------------
# Helper Functions
# -------------------------------------------------------------------
def estimate_eta(lat1, lon1, lat2, lon2, speed=40):
    """
    Estimate ETA and distance between two coordinates.

    Returns:
        eta_minutes (float), distance_km (float)
    """
    distance_km = geodesic((lat1, lon1), (lat2, lon2)).km
    eta_minutes = (distance_km / speed) * 60
    return round(eta_minutes, 1), round(distance_km, 2)

def generate_map(lat1, lon1, lat2, lon2, vehicle_id):
    """
    Generate an HTML map showing rerouted vehicle path.
    """
    fmap = folium.Map(location=[lat1, lon1], zoom_start=13)

    folium.Marker(
        [lat1, lon1],
        icon=folium.Icon(color="red", icon="truck", prefix="fa"),
        tooltip=f"Vehicle {vehicle_id} (Rerouted)"
    ).add_to(fmap)

    folium.Marker(
        [lat2, lon2],
        icon=folium.Icon(color="green"),
        tooltip="New Destination"
    ).add_to(fmap)

    folium.PolyLine(
        [[lat1, lon1], [lat2, lon2]],
        weight=3
    ).add_to(fmap)

    map_path = os.path.join(OUTPUT_DIR, f"reroute_{vehicle_id}.html")
    fmap.save(map_path)
    return map_path

# -------------------------------------------------------------------
# Core Rerouting Logic
# -------------------------------------------------------------------
def run_dynamic_rerouting():
    """
    Executes one rerouting cycle:
    - Reads Phase-3 fleet schedule
    - Identifies vehicles flagged for rerouting
    - Computes alternate routes
    - Logs reroute events
    """

    if not os.path.exists(FLEET_FILE):
        print("[INFO] Phase-3 output not available. Waiting...")
        return

    fleet_df = pd.read_csv(FLEET_FILE)

    if "rerouted_at" not in fleet_df.columns:
        fleet_df["rerouted_at"] = ""

    reroute_count = 0

    for idx, row in fleet_df.iterrows():

        # Process only vehicles flagged by Phase-3
        if not row.get("reroute_required", False):
            continue

        # Prevent repeated rerouting
        if str(row["rerouted_at"]).strip():
            continue

        # Generate alternate destination
        new_lat = row["dest_lat"] + random.uniform(-MAX_DEST_SHIFT, MAX_DEST_SHIFT)
        new_lon = row["dest_lon"] + random.uniform(-MAX_DEST_SHIFT, MAX_DEST_SHIFT)

        eta, distance = estimate_eta(
            row["latitude"], row["longitude"], new_lat, new_lon
        )

        fleet_df.at[idx, "dest_lat"] = new_lat
        fleet_df.at[idx, "dest_lon"] = new_lon
        fleet_df.at[idx, "eta_min"] = eta
        fleet_df.at[idx, "rerouted_at"] = datetime.now(IST).strftime(
            "%Y-%m-%d %H:%M:%S"
        )

        # Log rerouting event
        log_entry = pd.DataFrame([{
            "vehicle_id": row["vehicle_id"],
            "risk_score": row["risk_score"],
            "reroute_reason": row["reroute_reason"],
            "new_eta_min": eta,
            "distance_km": distance,
            "rerouted_at": fleet_df.at[idx, "rerouted_at"]
        }])

        log_entry.to_csv(
            REROUTE_LOG,
            mode="a",
            header=not os.path.exists(REROUTE_LOG),
            index=False
        )

        map_path = generate_map(
            row["latitude"],
            row["longitude"],
            new_lat,
            new_lon,
            row["vehicle_id"]
        )

        print(
            f"[ALERT] Vehicle {row['vehicle_id']} rerouted | "
            f"ETA: {eta} min | Distance: {distance} km"
        )
        print(f"[INFO] Route visualization saved at: {map_path}")

        reroute_count += 1

    fleet_df.to_csv(FLEET_FILE, index=False)

    if reroute_count:
        print(f"[INFO] Phase-4 completed with {reroute_count} reroute(s).")
    else:
        print("[INFO] Phase-4 completed. No rerouting required.")

# -------------------------------------------------------------------
# Continuous Execution Loop
# -------------------------------------------------------------------
if __name__ == "__main__":
    print("[SYSTEM] Phase-4 Dynamic Rerouting service started.")

    try:
        while True:
            run_dynamic_rerouting()
            time.sleep(UPDATE_INTERVAL)
    except KeyboardInterrupt:
        print("\n[SYSTEM] Phase-4 service stopped by user.")


Writing phase4_dynamic_rerouting/dynamic_rerouting.py


# **PHASE-5 — MONITORING DASHBOARD**

This phase provides a real-time visualization layer for fleet managers
using Streamlit.


### **Fleet Monitoring Dashboard**
Displays:
- Live GPS tracking
- Incident predictions
- Traffic heatmaps
- Fleet utilization
- Fuel consumption forecasts
- Driver behavior alerts


In [44]:
%%writefile phase5_dashboard/dashboard.py
import streamlit as st
import pandas as pd
import folium
from folium.plugins import HeatMap
from streamlit_folium import st_folium
from streamlit_autorefresh import st_autorefresh
import os
import json

# =====================================================
# PAGE CONFIGURATION
# =====================================================
st.set_page_config(
    page_title="🚚 AI Fleet Monitoring System",
    layout="wide"
)

st.title("🚛 AI-Powered Fleet Optimization Dashboard")
st.caption("📡 Phase-5 | Real-Time Fleet Monitoring & Intelligence")

# =====================================================
# SESSION STATE INITIALIZATION
# =====================================================
st.session_state.setdefault("paused", False)
st.session_state.setdefault("refresh_count", 0)
st.session_state.setdefault("map_center", [17.3850, 78.4867])
st.session_state.setdefault("map_zoom", 10)
st.session_state.setdefault("demo_mode", True)

# =====================================================
# SIDEBAR NAVIGATION
# =====================================================
st.sidebar.header("🧭 Navigation")
page = st.sidebar.radio(
    "Select View 👇",
    [
        "📍 Live GPS & Routes",
        "🔥 Traffic Heatmap",
        "⚠️ Incident Predictions",
        "⛽ Fuel Forecast",
        "🚨 Driver Alerts",
        "📋 Fleet Assignments"
    ]
)

# =====================================================
# LIVE CONTROLS
# =====================================================
st.sidebar.header("🎛️ Live Controls")

auto_refresh = st.sidebar.toggle("🔄 Auto Refresh", value=True)

refresh_sec = st.sidebar.slider(
    "⏱️ Refresh Interval (seconds)",
    min_value=5,
    max_value=120,
    value=30,
    disabled=not auto_refresh
)

max_vehicles = st.sidebar.slider(
    "🚗 Active Vehicles",
    min_value=5,
    max_value=50,
    value=20,
    step=5
)

if st.sidebar.button("⏸️ Pause / ▶️ Resume"):
    st.session_state.paused = not st.session_state.paused

# =====================================================
# DEMO MODE CONTROL
# =====================================================
st.sidebar.header("🧪 Demo Controls")
st.session_state.demo_mode = st.sidebar.toggle(
    "🛣️ Demo Mode (Force Reroutes)",
    value=st.session_state.demo_mode
)

# =====================================================
# AUTO REFRESH HANDLING
# =====================================================
if auto_refresh and not st.session_state.paused:
    st_autorefresh(interval=refresh_sec * 1000, key="fleet_refresh")
    st.session_state.refresh_count += 1

fps = round(1 / refresh_sec, 2) if auto_refresh else 0

st.sidebar.markdown("---")
st.sidebar.metric("🔁 Updates", st.session_state.refresh_count)
st.sidebar.metric("⚡ Refresh Rate (FPS)", fps)
st.sidebar.metric(
    "🟢 System Status",
    "Paused ⏸️" if st.session_state.paused else "Running ▶️"
)

# =====================================================
# CUSTOM STYLING
# =====================================================
st.markdown("""
<style>
.metric-card {
    background-color: #ffffff;
    padding: 18px;
    border-radius: 14px;
    box-shadow: 0px 6px 18px rgba(0,0,0,0.12);
}
.metric-title {
    font-size: 14px;
    color: #6b7280;
    font-weight: 600;
}
.metric-value {
    font-size: 28px;
    font-weight: 800;
    color: #111827;
}
</style>
""", unsafe_allow_html=True)

# =====================================================
# FILE PATHS
# =====================================================
BASE_DIR = os.path.dirname(__file__)
DATA_DIR = os.path.join(BASE_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)

STREAM_FILE = os.path.join(DATA_DIR, "live_stream.csv")
PRED_FILE = os.path.join(DATA_DIR, "live_predictions.csv")
FLEET_FILE = os.path.join(DATA_DIR, "fleet_schedule.csv")
REROUTE_FILE = os.path.join(DATA_DIR, "rerouting_logs.csv")
CONFIG_FILE = os.path.join(DATA_DIR, "demo_config.json")

with open(CONFIG_FILE, "w") as f:
    json.dump({"DEMO_MODE": st.session_state.demo_mode}, f)

# =====================================================
# SAFE CSV LOADER
# =====================================================
def safe_read_csv(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame()
    try:
        return pd.read_csv(path, engine="python", on_bad_lines="skip")
    except Exception:
        return pd.DataFrame()

df_stream = safe_read_csv(STREAM_FILE)
df_pred = safe_read_csv(PRED_FILE)
df_fleet = safe_read_csv(FLEET_FILE)
df_reroute = safe_read_csv(REROUTE_FILE)

# =====================================================
# DATA PREPARATION
# =====================================================
latest_pos = (
    df_stream.groupby("vehicle_id").last().reset_index().head(max_vehicles)
    if not df_stream.empty else pd.DataFrame()
)

latest_pred = (
    df_pred.groupby("vehicle_id").last().reset_index().head(max_vehicles)
    if not df_pred.empty else pd.DataFrame()
)

for col in ["vehicle_id", "latitude", "longitude", "speed", "traffic_index"]:
    if col not in latest_pos.columns:
        latest_pos[col] = 0

for col in ["vehicle_id","accident_risk","breakdown_risk","slowdown_risk","route_blockage_risk"]:
    if col not in latest_pred.columns:
        latest_pred[col] = 0.0

latest_pos = latest_pos[
    (latest_pos["latitude"] != 0) & (latest_pos["longitude"] != 0)
]

# =====================================================
# KPI CARD COMPONENT
# =====================================================
def metric_card(container, title, value, icon):
    container.markdown(f"""
    <div class="metric-card">
        <div class="metric-title">{icon} {title}</div>
        <div class="metric-value">{value}</div>
    </div>
    """, unsafe_allow_html=True)

# =====================================================
# MAP GENERATION
# =====================================================
def generate_map():
    fmap = folium.Map(
        location=st.session_state.map_center,
        zoom_start=st.session_state.map_zoom,
        control_scale=True
    )

    for _, row in latest_pos.iterrows():
        risk_series = latest_pred.loc[
            latest_pred["vehicle_id"] == row["vehicle_id"],
            "accident_risk"
        ]
        risk = float(risk_series.values[0]) if not risk_series.empty else 0.0
        color = "green" if risk < 0.4 else "orange" if risk < 0.75 else "red"

        folium.Marker(
            [row["latitude"], row["longitude"]],
            popup=f"""
            🚚 <b>Vehicle:</b> {row['vehicle_id']}<br>
            ⚡ <b>Speed:</b> {row['speed']} km/h<br>
            ⚠️ <b>Accident Risk:</b> {risk:.2f}
            """,
            icon=folium.Icon(color=color, icon="truck", prefix="fa")
        ).add_to(fmap)

    return fmap

# =====================================================
# PAGE RENDERING
# =====================================================
if page == "📍 Live GPS & Routes":
    st.subheader("📡 Live Vehicle Tracking")

    map_data = st_folium(
        generate_map(),
        width=1300,
        height=550,
        returned_objects=["center", "zoom"]
    )

    
    if map_data:
        if map_data.get("center"):
            st.session_state.map_center = [
                map_data["center"]["lat"],
                map_data["center"]["lng"]
            ]
        if map_data.get("zoom"):
            st.session_state.map_zoom = map_data["zoom"]

elif page == "🔥 Traffic Heatmap":
    st.subheader("🔥 Traffic Density Heatmap")
    heat_map = folium.Map(location=st.session_state.map_center, zoom_start=st.session_state.map_zoom)
    HeatMap(latest_pos[["latitude","longitude","traffic_index"]].values.tolist(), radius=20).add_to(heat_map)
    st_folium(heat_map, width=1300, height=520)


elif page == "⚠️ Incident Predictions":
    # =====================================================
    # AVERAGE INCIDENT PREDICTIONS (FLEET LEVEL)
    # =====================================================
    st.subheader("📈 Average Incident Predictions")
    if latest_pred.empty:
        st.info("No prediction data available")
    else:
        a1, a2, a3, a4 = st.columns(4)
        avg_accident = latest_pred["accident_risk"].mean() * 100
        avg_breakdown = latest_pred["breakdown_risk"].mean() * 100
        avg_slowdown = latest_pred["slowdown_risk"].mean() * 100
        avg_blockage = latest_pred["route_blockage_risk"].mean() * 100
        metric_card(a1, "Accident Risk", f"{avg_accident:.1f}%", "🚨")
        metric_card(a2, "Breakdown Risk", f"{avg_breakdown:.1f}%", "🛠")
        metric_card(a3, "Slowdown Risk", f"{avg_slowdown:.1f}%", "🐢")
        metric_card(a4, "Route Blockage Risk", f"{avg_blockage:.1f}%", "⛔")

    st.subheader("📊 Incident Risk Predictions")
    st.dataframe(latest_pred, use_container_width=True)


elif page == "⛽ Fuel Forecast":
    st.subheader("⛽ Fuel Consumption Forecast")
    latest_pos["fuel_estimate"] = latest_pos["speed"]*0.05 + latest_pos["traffic_index"]*2
    st.line_chart(latest_pos.set_index("vehicle_id")["fuel_estimate"])

elif page == "🚨 Driver Alerts":
    st.subheader("🚨 Driver Alerts (Action Required)")

    if latest_pos.empty:
        st.info("No alerts available at this time.")
    else:
        # Merge live GPS data with prediction data
        alerts = latest_pos.merge(
            latest_pred,
            on="vehicle_id",
            how="left"
        )

        # Fill missing prediction values safely
        alerts[[
            "accident_risk",
            "breakdown_risk",
            "slowdown_risk",
            "route_blockage_risk"
        ]] = alerts[[
            "accident_risk",
            "breakdown_risk",
            "slowdown_risk",
            "route_blockage_risk"
        ]].fillna(0.0)

        # Default severity
        alerts["severity"] = "LOW"

        # Medium severity conditions
        alerts.loc[
            alerts["traffic_index"] > 1.3,
            "severity"
        ] = "MEDIUM"

        # High severity conditions (critical)
        alerts.loc[
            (alerts["traffic_index"] > 1.5) |
            (alerts["speed"] > 90) |
            (alerts["accident_risk"] > 0.75),
            "severity"
        ] = "HIGH"

        # Filter only actionable alerts
        alerts = alerts[alerts["severity"] != "LOW"]

        # KPI: Active alerts count
        st.metric("🚨 Active Alerts", len(alerts))

        if alerts.empty:
            st.success("No critical driver alerts 🎉")
        else:
            # Human-friendly alert labels
            alerts["alert"] = alerts["severity"].map({
                "MEDIUM": "🟠 Caution",
                "HIGH": "🔴 Immediate Action"
            })

            # Display alerts sorted by risk priority
            st.dataframe(
                alerts[
                    [
                        "vehicle_id",
                        "speed",
                        "traffic_index",
                        "accident_risk",
                        "alert"
                    ]
                ].sort_values(
                    by="accident_risk",
                    ascending=False
                ),
                use_container_width=True
            )

            st.warning(
                "⚠️ High-risk alerts require immediate attention to prevent accidents or delays."
            )

elif page == "📋 Fleet Assignments":
    st.subheader("📋 Fleet Schedule & Assignments")
    st.dataframe(df_fleet, use_container_width=True)

# =====================================================
# FLEET SUMMARY
# =====================================================
st.subheader("📊 Fleet Summary")

c1, c2, c3, c4 = st.columns(4)

metric_card(c1, "Active Vehicles", latest_pos["vehicle_id"].nunique(), "🚗")
metric_card(c2, "High-Risk Vehicles", (latest_pred["accident_risk"] >= 0.75).sum(), "⚠️")
metric_card(c3, "Fleet Utilization", "Optimized ✅", "📈")
metric_card(c4, "Reroutes Triggered", len(df_reroute), "🛣️")

st.success("✅ Phase-5 Fleet Monitoring Dashboard is running successfully 🚀")


Overwriting phase5_dashboard/dashboard.py
